In [ ]:
# pip install bs4
# pip install selenium
# pip install requests
# pip install webdriver-manager

https://medium.com/analytics-vidhya/web-scraping-google-search-results-with-selenium-and-beautifulsoup-4c534817ad88

In [ ]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
# Import the beautifulsoup and request libraries of python.
import requests
import bs4

In [ ]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--headless")
driver = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=chrome_options)
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [ ]:
from ast import Num
from magic_google import MagicGoogle
import pprint

PROXIES = None
# PROXIES = [{
#     'http': 'http://192.168.2.207:1080',
#     'https': 'http://192.168.2.207:1080'
# }]

# Or MagicGoogle()
mg = MagicGoogle(PROXIES)
links=[]
# Crawling url
for url in mg.search_url(query='site:upenn.edu "innovation"', num=1000):
    links.append(url)
    

In [ ]:
import pandas as pd
ECOSYSTEM_PENN_LIST = [
'https://www.sas.upenn.edu/',
'https://venturelab.upenn.edu/',
'https://www.asc.upenn.edu/',
'https://www.dental.upenn.edu/',
'https://www.design.upenn.edu/',
'https://www.gse.upenn.edu/',
'https://www.seas.upenn.edu/',
'https://www.law.upenn.edu/',
'https://www.med.upenn.edu/',
'https://www.nursing.upenn.edu/',
'https://www.sp2.upenn.edu/',
'https://www.vet.upenn.edu/',
'https://pci.upenn.edu/',
'https://www.library.upenn.edu/',
'https://www.chop.edu/',
'https://sciencecenter.org/',
'https://wistar.org/',
'https://www.upenn.edu/research-and-innovation',
'https://research.upenn.edu/',
'https://research.upenn.edu/centers-and-institutes/',
'https://www.pennovation.upenn.edu/the-community#innovators',
'https://penntoday.upenn.edu/subtopic/innovation',
'https://yprize.upenn.edu/']
df5 = pd.DataFrame(ECOSYSTEM_PENN_LIST)

In [ ]:
import pandas as pd
df1 = pd.read_csv('GoogleSearchData/siteupenn.edu innovation (1).csv',delimiter=';')
df2 = pd.read_csv('GoogleSearchData/siteupenn.edu innovation (2).csv',delimiter=';')
df3 = pd.read_csv('GoogleSearchData/siteupenn.edu innovation (3).csv',delimiter=';')
df4 = pd.read_csv('GoogleSearchData/siteupenn.edu innovation (4).csv',delimiter=';')
df1 = df1.Url
df2 = df2.Url
df3 = df3.Url
df4 = df4.Url

In [ ]:
from dataclasses import replace
df = pd.concat([df1,df2,df3,df4,df5])
df = df.reset_index().drop(columns=['index'])
df.drop_duplicates(inplace=True)
df.rename(columns={0:'url'}, inplace=True)
df.to_csv('total_links.csv')

In [ ]:
import pandas as pd
links = pd.read_csv('total_links.csv', index_col=0)

In [ ]:
links.head()

In [ ]:
sum(links.url.duplicated()) # check for duplication

In [ ]:
len(links)

In [ ]:
print(links[links.url.str.endswith("pdf")])

In [ ]:
# exclude pdf links
links = links[~links.url.str.endswith("pdf")]

In [ ]:
domains = []
for link in links.url:
    link_element = link.split('/')
    domains.append("https://" + link_element[2].replace("www.", ""))

In [ ]:
links['domain'] = domains

In [ ]:
len(list(set(domains)))

In [ ]:
links

In [ ]:
page = requests.get('https://upenn.edu', verify=False)
count = sum(page.text.lower().count(x) for x in ('innovation','innovative','innovat'))
count

In [ ]:
page.text.lower()

In [ ]:
import requests
innovate_count = []
for i in links.url:
    page = requests.get(i, verify=False)
    count = sum(page.text.lower().count(x) for x in ('innovation','innovative','innovat'))
    innovate_count.append(count)
innovate_count

In [ ]:
links['innovate_count'] = innovate_count

In [ ]:
links[links['innovate_count']==0]

Collapsed spreadsheet: domains, Col: how many pages are there, and how many times innovation is mentioned

In [ ]:
links.groupby('domain')['url'].count()

In [ ]:
groupby_domain = pd.DataFrame(links.groupby('domain')['url'].count())

In [ ]:
groupby_domain['innovate_count'] = links.groupby('domain')['innovate_count'].sum()

In [ ]:
groupby_domain.rename(columns={'url':'page_count'}, inplace=True)

In [ ]:
groupby_domain[groupby_domain['innovate_count']==0]

site-site matrix

In [ ]:
with pd.ExcelWriter('site_domainV2.xlsx') as writer:  
    links.to_excel(writer, sheet_name='site', index=False)
    groupby_domain.to_excel(writer, sheet_name='domain')

In [ ]:
destination_dir = 'home/mgmt/wanxing/html_data/'

In [ ]:
import urllib.request
import os
link = links.url[2]
domain_name=[]
print(link)
link_element = link.split('/')
link_element[2] = link_element[2].replace("www.","")
reversed_element  = link_element[2].split('.')[::-1]
print(reversed_element)
reversed_element = ['.'.join(reversed_element[:i][::-1]) for i in range(1,len(reversed_element)+1)]
print(reversed_element)
link_element[2] = '/'.join(reversed_element)
print(len(link_element))
if len(link_element) <= 4:
    domain_name.append(link_element[2])
else:
    for i in range(2, len(link_element)-2):
        if len(link_element[i]) != 0:
            domain_name.append(link_element[i])
html_name = link_element[-2]
# print(link_element)
domain_name = '/'.join(domain_name)
print(domain_name)
print(link)
filepath = '/Users/wanxing/PycharmProjects/PennEcosystem/html_data/c3i'
filepath = os.path.join(destination_dir, domain_name)
print(filepath)
if not os.path.exists(filepath):
    if not os.path.exists(destination_dir):
        os.makedirs(destination_dir)
    os.makedirs(filepath)
filename = os.path.join(filepath, html_name+".html")
urllib.request.urlretrieve(link, filename)

In [ ]:

lst= requests.get(url[0], verify=False)
lst= sum(lst.text.count(x) for x in ('innovation', 'innovations', 'innovate'))

In [ ]:
import urllib.request
import os

destination_dir = 'home/mgmt/wanxing/html_data/'
# print(link)
for link in links:
    domain_name=[]
    link_element = link.split('/')
    link_element[2] = link_element[2].replace("www.","").replace(".edu","")
    reversed_element  = link_element[2].split('.')[::-1]
    link_element[2] = '/'.join(reversed_element)
    for i in range(2, len(link_element)-2):
        if len(link_element[i]) != 0:
            domain_name.append(link_element[i])
    name = link_element[-2]
    # print(link_element)
    domain_name = '/'.join(domain_name)
    # print(domain)
    # print(link)
    # filepath = '/Users/wanxing/PycharmProjects/PennEcosystem/html_data/c3i'
    filepath = os.path.join(destination_dir, domain_name)
    # print(filepath)
    if not os.path.exists(filepath):
        if not os.path.exists(destination_dir):
            os.makedirs(destination_dir)
        os.makedirs(filepath)
    filename = os.path.join(filepath, name+".html")
    try:
        urllib.request.urlretrieve(link, filename)
    except:
        continue
    # print(filename)
    print("Downloading "+name)
    time.sleep(5)

In [ ]:
## Google search
import time
import urllib.request
import os

text= "innovation"
domain = "q=site%3Aupenn.edu+"
links = [] # Initiate empty list to capture final results
destination_dir = 'smb://hpcc.wharton.upenn.edu/wanxing/html_data/'
# Specify number of pages on google search, each page contains 10 #links

n_pages = 14200
for page in range(1, n_pages):
    # print(page)
    url = "https://www.google.com/search?" + domain + text + "&start=" + str((page - 1) * 10)
    # print(url)
    driver.get(url)
    soup = bs4.BeautifulSoup(driver.page_source, 'html.parser')
    # print(soup)
    search = soup.find_all('div', class_="yuRUbf")
    # print(search)
    for h in search:
        links.append(h.a.get('href'))
    for h in search:
        domain=[]
        link = h.a.get('href')
        # print(link)
        if link is not None:
            link_element = link.split('/')
            link_element[2] = link_element[2].replace("www.","").replace(".upenn.edu","")
            for i in range(2, len(link_element)-2):
                if len(link_element[i]) != 0:
                    domain.append(link_element[i])
            name = link_element[-2]
            # print(link_element)
            domain = '/'.join(domain)
            title = h.a.text
            links.append(link)
            # print(domain)
            # print(link)
            # filepath = '/Users/wanxing/PycharmProjects/PennEcosystem/html_data/c3i'
            filepath = os.path.join(destination_dir, domain)
            # print(filepath)
            if not os.path.exists(filepath):
                if not os.path.exists(destination_dir):
                    os.makedirs(destination_dir)
                os.makedirs(filepath)
            filename = os.path.join(filepath, name+".html")
            try:
                urllib.request.urlretrieve(link, filename)
            except:
                continue
            # print(filename)
            print("Downloading "+title)
            time.sleep(10)

In [ ]:
links

In [ ]:
import pandas as pd
links = pd.DataFrame(links, columns=['link'])
links.to_csv('links.csv')

In [ ]:
# import urllib.request
# import os
# # Penn Website search https://www.upenn.edu/search
# text= "innovation"
# links = [] # Initiate empty list to capture final results
# destination_dir = '/Users/wanxing/PycharmProjects/PennEcosystem/html_data/'
# # Specify number of pages on google search, each page contains 10 #links

# n_pages = 20
# for page in range(1, n_pages):
#    #https://www.upenn.edu/search?as_q=innovation#gsc.tab=0&gsc.q=innovation&gsc.page=1 
#     url = "https://www.upenn.edu/search?as_q=" + text + "#gsc.tab=0&gsc.q=innovation&gsc.page=" + str((page))
#     driver.get(url)
#     soup = bs4.BeautifulSoup(driver.page_source, 'html.parser')
#     search = soup.find_all('div', class_="gsc-thumbnail-inside")
#     for h in search:
#         domain=[]
#         link = h.a.get('href')
#         # print(link)
#         if link is not None:
#             link_element = link.split('/')
#             for i in range(2, len(link_element)):
#                 if len(link_element[i]) != 0:
#                     domain.append(link_element[i])
#             domain = '/'.join(domain)
#             title = h.a.text
#             links.append(link)
#             print(page)
#             print(domain)
#             print(link)

#             filepath = os.path.join(destination_dir, domain)
#             print(filepath)
#             if not os.path.exists(filepath):
#                 if not os.path.exists(destination_dir):
#                     os.makedirs(destination_dir)
#                 os.makedirs(filepath)
#             try:
#                 urllib.request.urlretrieve(link, filepath+'.html')
#             except:
#                 continue
#             print("Downloading "+title)

In [ ]:
#   # Make two strings with default google search URL
# # 'https://google.com/search?q=' and
# # our customized search keyword.
# # Concatenate them
# text= "innovation"
# domain = "site%3Aupenn.edu+"
# url = 'https://google.com/search?q=' + domain + text
  
# # Fetch the URL data using requests.get(url),
# # store it in a variable, request_result.
# request_result=requests.get( url )
  
# # Creating soup from the fetched request
# soup = bs4.BeautifulSoup(request_result.text,
#                          "html.parser")
# print(soup)

In [ ]:
# # soup.find.all( h3 ) to grab 
# # all major headings of our search result,
# heading_object=soup.find_all( 'h3' )
  
# # Iterate through the object 
# # and print it as a string.
# for info in heading_object:
#     print(info.getText())
#     print("------")